# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 12:09PM,246718,10,0085991161,ISDIN Corporation,Released
1,Sep 19 2022 12:09PM,246718,10,0085991163,ISDIN Corporation,Released
2,Sep 19 2022 12:09PM,246718,10,0085993164,ISDIN Corporation,Released
3,Sep 19 2022 12:09PM,246718,10,0085993166,ISDIN Corporation,Released
4,Sep 19 2022 12:09PM,246718,10,0085993172,ISDIN Corporation,Released
5,Sep 19 2022 12:09PM,246718,10,0085993191,ISDIN Corporation,Released
6,Sep 19 2022 12:09PM,246718,10,0085993189,ISDIN Corporation,Released
7,Sep 19 2022 12:05PM,246719,10,MSP214053,"Methapharm, Inc.",Released
8,Sep 19 2022 12:05PM,246719,10,MSP214065,"Methapharm, Inc.",Released
9,Sep 19 2022 11:50AM,246710,15,ALIFBA16W7ZKGYN,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,246708,Released,9
16,246709,Released,1
17,246710,Released,3
18,246718,Released,7
19,246719,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246708,NaN,9.0
246709,NaN,1.0
246710,NaN,3.0
246718,NaN,7.0
246719,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246677,1.0,0.0
246693,0.0,1.0
246697,10.0,9.0
246700,2.0,16.0
246701,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246677,1,0
246693,0,1
246697,10,9
246700,2,16
246701,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246677,1,0
1,246693,0,1
2,246697,10,9
3,246700,2,16
4,246701,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246677,1,
1,246693,,1
2,246697,10,9
3,246700,2,16
4,246701,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation
7,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc."
9,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc."
12,Sep 19 2022 11:37AM,246709,19,"NAPP Technologies, LLC"
13,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.
22,Sep 19 2022 11:33AM,246707,19,"GUSA Granules USA, Inc."
23,Sep 19 2022 11:33AM,246706,10,Emerginnova
24,Sep 19 2022 11:32AM,246705,10,ISDIN Corporation
43,Sep 19 2022 11:31AM,246704,10,"Snap Medical Industries, LLC"
59,Sep 19 2022 11:28AM,246703,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation,,7
1,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc.",,2
2,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc.",,3
3,Sep 19 2022 11:37AM,246709,19,"NAPP Technologies, LLC",,1
4,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.,,9
5,Sep 19 2022 11:33AM,246707,19,"GUSA Granules USA, Inc.",,1
6,Sep 19 2022 11:33AM,246706,10,Emerginnova,,1
7,Sep 19 2022 11:32AM,246705,10,ISDIN Corporation,,19
8,Sep 19 2022 11:31AM,246704,10,"Snap Medical Industries, LLC",7,9
9,Sep 19 2022 11:28AM,246703,19,"GUSA Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation,7,
1,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc.",2,
2,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc.",3,
3,Sep 19 2022 11:37AM,246709,19,"NAPP Technologies, LLC",1,
4,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.,9,
5,Sep 19 2022 11:33AM,246707,19,"GUSA Granules USA, Inc.",1,
6,Sep 19 2022 11:33AM,246706,10,Emerginnova,1,
7,Sep 19 2022 11:32AM,246705,10,ISDIN Corporation,19,
8,Sep 19 2022 11:31AM,246704,10,"Snap Medical Industries, LLC",9,7
9,Sep 19 2022 11:28AM,246703,19,"GUSA Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation,7,
1,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc.",2,
2,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc.",3,
3,Sep 19 2022 11:37AM,246709,19,"NAPP Technologies, LLC",1,
4,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation,7.0,NaN
1,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc.",2.0,NaN
2,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc.",3.0,NaN
3,Sep 19 2022 11:37AM,246709,19,"NAPP Technologies, LLC",1.0,NaN
4,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.,9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 19 2022 12:09PM,246718,10,ISDIN Corporation,7.0,0.0
1,Sep 19 2022 12:05PM,246719,10,"Methapharm, Inc.",2.0,0.0
2,Sep 19 2022 11:50AM,246710,15,"Alliance Pharma, Inc.",3.0,0.0
3,Sep 19 2022 11:37AM,246709,19,"NAPP Technologies, LLC",1.0,0.0
4,Sep 19 2022 11:35AM,246708,10,Eywa Pharma Inc.,9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1973661,65.0,9.0
15,246710,3.0,0.0
16,493370,1.0,1.0
19,986821,11.0,15.0
20,246697,9.0,10.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1973661,65.0,9.0
1,15,246710,3.0,0.0
2,16,493370,1.0,1.0
3,19,986821,11.0,15.0
4,20,246697,9.0,10.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,65.0,9.0
1,15,3.0,0.0
2,16,1.0,1.0
3,19,11.0,15.0
4,20,9.0,10.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,65.0
1,15,Released,3.0
2,16,Released,1.0
3,19,Released,11.0
4,20,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,9.0,0.0,1.0,15.0,10.0
Released,65.0,3.0,1.0,11.0,9.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,9.0,0.0,1.0,15.0,10.0
1,Released,65.0,3.0,1.0,11.0,9.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,9.0,0.0,1.0,15.0,10.0
1,Released,65.0,3.0,1.0,11.0,9.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()